# importind library

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from bs4 import BeautifulSoup
import urllib 
from urllib import request
import requests
import re
import json
import pprint as pp
import os
import random

# reading data

In [2]:
total = pd.read_csv('trainee_total.csv', encoding = 'cp949', index_col = 0)
s1 = total.loc[total['season']== 1, : ]
s2 = total.loc[total['season']== 2, : ]

# API

- 시즌에 맞게 데이터와 기간을 설정하여 추출한다.

## 날짜 할당

In [3]:
s1_dt_idx = pd.date_range(start = '20160129', end = '20160218') 
s1_dt = s1_dt_idx.strftime('%Y-%m-%d').tolist()

## API 요청

In [4]:
key = open('naver_api.json', 'r').read()
naver_api = json.loads(key)

In [5]:
def get_trend(body):
    url = "https://openapi.naver.com/v1/datalab/search"
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    request.add_header("Content-Type","application/json")
    response = urllib.request.urlopen(request, data=body.encode("utf-8"))

    return json.loads(response.read().decode('utf-8'))

In [6]:
# 할당 받은 api의 id, secret 넣기
i = 0
client_id = naver_api.get('groups').get('info')[i].get('client_id')
client_secret = naver_api.get('groups').get('info')[i].get('client_secret')

## API 돌리기

In [7]:
tmp_dt = s1_dt

In [8]:
req_list = []
result_dict = {'date' : [], 'name' : [], 'ratio' : [], 'div' : []}


for dt in tmp_dt : 
    
    s1 = s1.sample(frac=1).reset_index(drop=True) # 5명 랜덤 배정을 위해
    
    for idx in range(0, len(s1), 5):
        
        slc = s1.iloc[idx : idx + 5]
        
        req_list = {


                'timeUnit' : 'date',
                'startDate' : dt,
                'endDate' : dt,
                'keywordGroups' : [
                    {
                        'groupName' : slc.loc[i, 'name'],
                        'keywords' : [slc.loc[i, 'name']]
                    }

                    for i in slc.index
                ]
            }
        
        
        tmp_api = json.dumps(req_list, indent = 4, ensure_ascii = False)
        tmp_trend = get_trend(tmp_api)
        
        
        for i in range(len(tmp_trend.get('results'))) :
            try : 
                result_dict['name'].append(tmp_trend.get('results')[i].get('title'))
                result_dict['div'].append(idx/5)
                result_dict['date'].append(tmp_trend.get('results')[i].get('data')[0].get('period'))
                result_dict['ratio'].append(tmp_trend.get('results')[i].get('data')[0].get('ratio'))


            except IndexError : 
                result_dict['date'].append('null')
                result_dict['ratio'].append('null')
            tmp_df = pd.DataFrame(result_dict)
        

In [9]:
tmp_df.to_csv('s1_rt01_0129-0218.csv', encoding = 'cp949')

In [128]:
s1_rt01_01 = pd.read_csv('s1_rt01_0101-0128.csv', encoding = 'cp949', index_col = 0)
s1_rt01_02 = pd.read_csv('s1_rt01_0129-0218.csv', encoding = 'cp949', index_col = 0)

In [129]:
s1_rt01 = pd.concat([s1_rt01_01, s1_rt01_02], axis = 0, ignore_index=True) # 두번에 나누어 저장한 데이터를 합침

## nan처리

In [130]:
for i in range(len(s1_rt01)) :     # ratio nan = 0
    if np.isnan(s1_rt01['ratio'][i]) == True :
        s1_rt01['ratio'][i] = 0

/Users/juu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [131]:
for i in range(len(s1_rt01)) :         # foward fill date
    s1_rt01['date'].ffill(axis = 0, inplace = True)

## ratio 반올림

In [132]:
s1_rt01['ratio'] = round(s1_rt01['ratio'])

## 추출

In [133]:
s1_rt01.to_csv('s1_rt01.csv', encoding = 'cp949') # 최종 rt01